In [1]:
import json
import numpy as np
import pandas as pd
import math
import re

## Готовим данные

In [2]:
fields = [
    l for l in open('data_description.txt', 'r').read().split('\n')
    if len(l.strip()) > 0
]

In [3]:
columns = [f.strip().split(': ') for f in fields if not f.startswith(' ')]
col_names = {c[0]: f"{c[0]}: {c[1]}" for c in columns}
col_names['SalePrice'] = 'SalePrice'

In [4]:
last_key = None
fv = {}

for f in fields:
    if not f.startswith(' '):
        last_key = f.split(': ')[0]
        continue
    value_pair = f.strip().split('\t')
    fv[(last_key, value_pair[0])] = value_pair[1]

In [5]:
s = pd.read_csv('train.csv')

In [6]:
s

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [7]:
row_dict = s.iloc[0]

facts = []
for k in row_dict.keys():
    if k not in col_names:
        continue
    if str(row_dict[k]) == 'nan':
        continue
    facts.append(col_names[k])
    if (k, str(row_dict[k])) in fv:
        facts.append(f'- {fv[(k, str(row_dict[k]))]}')
    else:
        facts.append(f'- {row_dict[k]}')
        
request = '\n'.join(facts).replace('\n- ', ': ')
request = re.sub('\n[^:]+: (?:Identifies )?', '\n', request)
request = re.sub('\n(\d+)', '\nPrice: $\\1', request)

In [8]:
print(request)

MSSubClass: Identifies the type of dwelling involved in the sale.: 2-STORY 1946 & NEWER
the general zoning classification of the sale.: Residential Low Density
Linear feet of street connected to property: 65.0
Lot size in square feet: 8450
Type of road access to property: Paved
General shape of property: Regular
Flatness of the property: Near Flat/Level
Type of utilities available: All public Utilities (E,G,W,& S)
Lot configuration: Inside lot
Slope of property: Gentle slope
Physical locations within Ames city limits: College Creek
Proximity to various conditions: Normal
Proximity to various conditions (if more than one is present): Normal
Type of dwelling: Single-family Detached
Style of dwelling: Two story
Rates the overall material and finish of the house: Good
Rates the overall condition of the house: Average
Original construction date: 2003
Remodel date (same as construction date if no remodeling or additions): 2003
Type of roof: Gable
Roof material: Standard (Composite) Shingle
E

[Yandex Cloud	](https://console.cloud.yandex.ru/folders/b1g7pe1dpqkubo7gpr9u?utm_source=emailing&utm_campaign=links&utm_content=console&utm_id=4089aa62-daa9-5ae0-ed07-5b3307b4706a&utm_medium=mail&utm_term=1)

[Промт-режим YandexGPT](https://console.cloud.yandex.ru/folders/b1g7pe1dpqkubo7gpr9u/yandexgpt/instruct)

## Делаем запросы в АПИ

[OAuth-токен](https://oauth.yandex.ru/verification_code#access_token=y0_AgAAAAAUE_SCAATuwQAAAADzOqW8bcf8jkQTTUKGQrjbj0ilEXsYAAg&token_type=bearer&expires_in=31536000)

In [9]:
oauth_token = "y0_AgAAAAAUE_SCAATuwQAAAADzOqW8bcf8jkQTTUKGQrjbj0ilEXsYAAg"

In [10]:
!curl -d "{\"yandexPassportOauthToken\":\"y0_AgAAAAAUE_SCAATuwQAAAADzOqW8bcf8jkQTTUKGQrjbj0ilEXsYAAg\"}" "https://iam.api.cloud.yandex.net/iam/v1/tokens"

{

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   405  100   316  100    89    558    157 --:--:-- --:--:-- --:--:--   716



 "iamToken": "t1.9euelZrKxsqRlJrKzYuWlcmcjs2OjO3rnpWals3Ky5aOmJOZjJzOj5yKm57l9PcLLD1U-e82dxr23fT3S1o6VPnvNnca9s3n9euelZqajZbJzpKZlczIkZieis6Wy-_8xeuelZqajZbJzpKZlczIkZieis6Wyw.bd-mZQirp970j_GhnEIj-edHnViB0ZmNcdDOMO8qHOli99gTUUiYheo_uQBQly1UqvPO1c8EUN5olDBBNZWhAQ",
 "expiresAt": "2023-12-07T05:05:56.020530249Z"
}


In [16]:
iamtoken = "t1.9euelZrKxsqRlJrKzYuWlcmcjs2OjO3rnpWals3Ky5aOmJOZjJzOj5yKm57l9PcLLD1U-e82dxr23fT3S1o6VPnvNnca9s3n9euelZqajZbJzpKZlczIkZieis6Wy-_8xeuelZqajZbJzpKZlczIkZieis6Wyw.bd-mZQirp970j_GhnEIj-edHnViB0ZmNcdDOMO8qHOli99gTUUiYheo_uQBQly1UqvPO1c8EUN5olDBBNZWhAQ"

In [17]:
dir_id = "b1g7pe1dpqkubo7gpr9u"

In [31]:
# import pickle
# [iamtoken, dir_id] = pickle.load(open('cred.pki', 'rb'))

In [18]:
import requests

In [19]:
result = requests.post(
    url='https://llm.api.cloud.yandex.net/llm/v1alpha/instruct',
    headers={'Authorization': f'Bearer {iamtoken}', 'x-folder-id': dir_id},
    json={
      "model": "general",
      "instruction_text": "Ниже перечислены характеристики недвижимости.",
      "request_text": f"{request}\n\nНа основании этой информации о недвижимости ниже напиши продающий текст на русском. Текст не должен содержать лжи и полностью соответствовать данным из характеристик.",
      "generation_options": {
        "max_tokens": 1500,  
        "temperature": 0.5
      }
    }
)

result

<Response [200]>

In [20]:
print(result.json()['result']['alternatives'][0]['text'])

Заголовок: Двухэтажный дом 2003 года постройки в пригороде

Описание: Этот уютный двухэтажный дом находится в престижном районе города, недалеко от колледжа. Он имеет общую площадь 1960 квадратных футов и расположен на участке площадью 8450 квадратных футов. Дом имеет все необходимые коммуникации, включая электричество, газ и воду.

Дом имеет две спальни, две ванные комнаты и просторную гостиную. В нем также есть гараж на две машины и большая веранда. Состояние дома хорошее, но можно сделать косметический ремонт.

Цена: $208 500. Это отличное предложение для тех, кто ищет уютный дом в тихом районе. Звоните и приходите на просмотр!


[Фундаментальные модели -> YandexGPT](https://datasphere.yandex.ru/foundation-models/ygpt)

## Fewshot

In [21]:
import pickle
train_dataset = pickle.load(open('fewshot.pkl', 'rb'))

In [68]:
# train_dataset

In [22]:
print("\n------\n\n".join(v['request'] + "\n-----\n" + v['response'] for v in train_dataset))

Ниже перечислены характеристики недвижимости.
MSSubClass: Identifies the type of dwelling involved in the sale.: 1-1/2 STORY FINISHED ALL AGES
the general zoning classification of the sale.: Residential Low Density
Linear feet of street connected to property: 85.0
Lot size in square feet: 14115
Type of road access to property: Paved
General shape of property: Slightly irregular
Flatness of the property: Near Flat/Level
Type of utilities available: All public Utilities (E,G,W,& S)
Lot configuration: Inside lot
Slope of property: Gentle slope
Physical locations within Ames city limits: Mitchell
Proximity to various conditions: Normal
Proximity to various conditions (if more than one is present): Normal
Type of dwelling: Single-family Detached
Style of dwelling: One and one-half story: 2nd level finished
Rates the overall material and finish of the house: Average
Rates the overall condition of the house: Average
Original construction date: 1993
Remodel date (same as construction date if n

## Finetune

[squad](https://www.kaggle.com/code/sawarn69/squad-stanford-q-a-json-to-pandas-dataframe)

In [47]:
df = pd.read_json('SQuAD.json', lines=True)

In [48]:
df.shape

(500, 6)

In [50]:
df[:3]

,ans_starts,ans_texts,context,id,question,title
0,[427],[mobile phones],The difference in the above factors for the ca...,57306bf68ab72b1400f9c4dc,What is one use that would require an antenna ...,Antenna_(radio)
1,[1020],[300],The coronation of Charlemagne as emperor on Ch...,56fb69e38ddada1400cd63f7,About how many counts existed in the Carolingi...,Middle_Ages
2,[339],[fossil pollen deposits in sediments],Plant responses to climate and other environme...,5726ccf9f1498d1400e8eb95,How can climate changes be determined from soil?,Botany


In [51]:
df['request'] = df.context + '\n\nQuestion: ' + df.question

In [52]:
df['response'] = df.ans_texts.str[0]

In [53]:
df[:3]

,ans_starts,ans_texts,context,id,question,title,request,response
0,[427],[mobile phones],The difference in the above factors for the ca...,57306bf68ab72b1400f9c4dc,What is one use that would require an antenna ...,Antenna_(radio),The difference in the above factors for the ca...,mobile phones
1,[1020],[300],The coronation of Charlemagne as emperor on Ch...,56fb69e38ddada1400cd63f7,About how many counts existed in the Carolingi...,Middle_Ages,The coronation of Charlemagne as emperor on Ch...,300
2,[339],[fossil pollen deposits in sediments],Plant responses to climate and other environme...,5726ccf9f1498d1400e8eb95,How can climate changes be determined from soil?,Botany,Plant responses to climate and other environme...,fossil pollen deposits in sediments


In [54]:
df = df.sample(df.shape[0])

In [55]:
data = [{'request': t.request, 'response': t.response} for t in df[['request', 'response']].itertuples()]

In [56]:
train = data[:400]

In [57]:
train = json.load(open('train.json', 'r'))

In [58]:
sample = df[~df.request.isin([t['request'] for t in train])].sample().iloc[0]
print(sample['request'])
print('-----')
print(sample['response'])

Before signing a developmental agreement with the Kane County Cougars in 2012, the Cubs had a Class A minor league affiliation on two occasions with the Peoria Chiefs (1985–1995 and 2004–2012). Ryne Sandberg managed the Chiefs from 2006 to 2010. In the period between those associations with the Chiefs the club had affiliations with the Dayton Dragons and Lansing Lugnuts. The Lugnuts were often affectionately referred to by Chip Caray as "Steve Stone's favorite team." The 2007 developmental contract with the Tennessee Smokies was preceded by Double A affiliations with the Orlando Cubs and West Tenn Diamond Jaxx. On September 16, 2014 the Cubs announced a move of their top Class A affiliate from Daytona in the Florida State League to Myrtle Beach in the Carolina League for the 2015 season. Two days later, on the 18th, the Cubs signed a 4-year player development contract with the South Bend Silver Hawks of the Midwest League, ending their brief relationship with the Kane County Cougars an

[datasphere](https://datasphere.yandex.ru/projects)